<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/Sonion/IGI_final_09_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from datetime import datetime, timedelta
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot
import pandas as pd
import re
import altair as alt

import statistics as st
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as scs
from math import sqrt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import warnings
warnings.filterwarnings("ignore")

#  Prepare data

In [26]:
#path='/content/drive/My Drive/Data/Sonion/IGI material Membrane all/'
path='/content/drive/My Drive/Data/Sonion/Armature 3000S/' # Good
#path='/content/drive/My Drive/Data/Sonion/Drive pin E25/' # Not good
#path2='/content/drive/My Drive/Data/MMD 2018/'

In [27]:
# VERY IMPORTANT INFORMATION
#print('Please input number of feature')
#feature_num=7
#print('Please input number of sample inspection')
#sample_num=5
#print('Please input position of sample value')
#namefile='MMD2019'

In [31]:
import glob
#print(path)
all_files1=glob.glob(path + '*.xlsx')
all_files2=glob.glob(path + '*.xlsm')
all_files=all_files1+all_files2
#sort file in directory by reverse:
all_files = sorted(all_files, reverse = False)
print('number of files: ',len(all_files))

number of files:  37


In [32]:
material_name=all_files[-1]
material_name=material_name.split('/')[-1].split('.')[0]
material_name

'B227012074  50202-3011028 Armature 3000S IGIR-03314'

In [38]:
sheet_all={}
n=0
count_file=1
for filename in all_files:
  #print(filename)
  print('count_file: ',str(count_file) + '/' + str(len(all_files)))
  count_file+=1
  try:
    df = pd.read_excel(filename, index_col = None, header = 0)
    #df.to_excel('file'+str(n)+'.xlsx')
  except:
    print('File cannot be read: '+filename)

  # find begin and end col 
  # find postion row with Pos/items
  a=df[df.columns[0]].str.contains('Pos.')
  item_row=df[a==True].index[0]

  values_col=df.iloc[item_row,:] # specific row that contain dimension name
  #print(values_col)
  values_col.reset_index(drop=True,inplace=True)
  #begin_col=values_col[values_col=='         Pos.\Item'].index 
  begin_col=1 # bat dau tu cot thu 1 tro di tu vi tri Pos.\Item
  end_col=values_col[values_col.isnull() == True].index[0]
  #print('begin_col',begin_col)
  #print('end_col',end_col)

  dim_dict={} # add all value, USL, LSL, UCL... in sheet
  n+=1
  if n==23:n=0
  for name in df.columns[begin_col:end_col]:
    new_df=pd.DataFrame()

    # find begin value collect (check if value is numeric or not):
    value_begin_row=df[df[df.columns[0]].apply(lambda x: isinstance(x, int))].index[0]

    new_df['Value']=df[name][value_begin_row:] #19:
    new_df=new_df.reset_index(drop=True)
    #print(new_df)
    #print(new_df.applymap(np.isreal))
    non_nummeric_row=new_df[new_df.applymap(np.isreal).values==False].index[0] # App o cuoi dong value 
    new_df=new_df[:non_nummeric_row]
    new_df['Value']=new_df['Value'].astype('float32')
    #print(df)
    # Check Date: (find row and column contains date)
    for i in range(7,10):
      try:
        row_date=df[df[df.columns[i]].apply(lambda x: isinstance(x, datetime))].index[0]
        column_date=i
      except:
        continue
    #print(row_date,column_date)
    new_df['Date']=df[df.columns[column_date]][row_date]
    #print(new_df)
    #new_df['Date']=new_df['Date'].apply(lambda x: x.strftime("%Y %m %d %H"))
    new_df['Date']=pd.to_datetime(new_df['Date'])
    new_df['Date']=new_df['Date'].apply(lambda x : x+timedelta(hours=n)) # and hour from 1 to 23 to avoid duplicate
    
    # chuyen vi tri date ra phia ngoai cung, value vao phia trong
    cols = new_df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    new_df=new_df[cols]
    
    # Add USL, LSL, UCL, LCL, Mean:
    new_df['USL']=df[name][item_row+2] # max
    new_df['LSL']=df[name][item_row+3] # min
    #new_df['Nominal']=df[name][item_row+8] # min
    
    #UCL,LCL,nominal: (lấy 25 gia tri gần nhất thôi)
    #sigma=3
    #new_df['UCL']=new_df['Value'].mean() + new_df['Value'].std()*sigma
    #new_df['LCL']=new_df['Value'].mean() - new_df['Value'].std()*sigma
    #new_df['Mean']=new_df['Value'].mean()
    new_df[new_df.columns[1:]]=new_df[new_df.columns[1:]].astype('float32')
    dim_name=df[name][item_row]
    dim_dict[dim_name]=new_df
  sheet_all[filename]=dim_dict

count_file:  1/37
count_file:  2/37
count_file:  3/37
count_file:  4/37
count_file:  5/37
count_file:  6/37
count_file:  7/37
count_file:  8/37
count_file:  9/37
count_file:  10/37
count_file:  11/37
count_file:  12/37
count_file:  13/37
count_file:  14/37
count_file:  15/37
count_file:  16/37
count_file:  17/37
count_file:  18/37
count_file:  19/37
count_file:  20/37
count_file:  21/37
count_file:  22/37
count_file:  23/37
count_file:  24/37
count_file:  25/37
count_file:  26/37
count_file:  27/37
count_file:  28/37
count_file:  29/37
count_file:  30/37
count_file:  31/37
count_file:  32/37
count_file:  33/37
count_file:  34/37
count_file:  35/37
count_file:  36/37
count_file:  37/37


In [40]:
sheet_all

{'/content/drive/My Drive/Data/Sonion/Armature 3000S/B227010147 50202-3011028 Armature 3000S IGIR-03314.xlsx': {'AD':                  Date  Value   USL  LSL
  0 2019-01-26 01:00:00  0.811  0.82  0.8
  1 2019-01-26 01:00:00  0.809  0.82  0.8
  2 2019-01-26 01:00:00  0.808  0.82  0.8
  3 2019-01-26 01:00:00  0.809  0.82  0.8
  4 2019-01-26 01:00:00  0.810  0.82  0.8,
  'AI':                  Date  Value   USL   LSL
  0 2019-01-26 01:00:00  2.599  2.61  2.59
  1 2019-01-26 01:00:00  2.608  2.61  2.59
  2 2019-01-26 01:00:00  2.605  2.61  2.59
  3 2019-01-26 01:00:00  2.599  2.61  2.59
  4 2019-01-26 01:00:00  2.604  2.61  2.59,
  'AK':                  Date  Value   USL   LSL
  0 2019-01-26 01:00:00  1.444  1.46  1.44
  1 2019-01-26 01:00:00  1.442  1.46  1.44
  2 2019-01-26 01:00:00  1.441  1.46  1.44
  3 2019-01-26 01:00:00  1.445  1.46  1.44
  4 2019-01-26 01:00:00  1.444  1.46  1.44,
  'AL':                  Date  Value   USL   LSL
  0 2019-01-26 01:00:00  2.555  2.58  2.54
  1 2019-

# Base file

Base file is the latest excel file contain all latest dim, it will merge data with all dims in old excel file. If old excel file have dim that not occur in present, it will not appear

In [41]:
base_file=sheet_all[list(sheet_all.keys())[-1]] # lay file cuoi cung
first_dim=list(base_file.keys())[0]
num_sub_sample=len(base_file[first_dim])
print(num_sub_sample)

5


In [42]:
# concat all process in each weeks based on keys value:
for file_name in list(sheet_all.keys())[:-1]: # ko tính base week nên từ 1: 
  other_file=sheet_all[file_name]
  for dim_name in other_file.keys(): # dict all process
    for dim_name_base in base_file.keys(): # dict all process
        if dim_name_base==dim_name: 
          base_file[dim_name]=\
          pd.concat([base_file[dim_name],
                    other_file[dim_name]])
          base_file[dim_name]=base_file[dim_name].sort_values(by='Date').reset_index(drop=True)
    #print(process_name)


# Calculate UCL, LCL, Cp, Cpk, Ppk base on stdev 25 lates subgroup sample

In [43]:
constants={
    2:1.128,
    3:1.693,
    4:2.059,
    5:2.326,
    6:2.534,
    7:2.704,
    8:2.847, 
    9: 2.970,
    10: 3.078,
    11: 3.173,
    12: 3.258,
    13: 3.336,
    14: 3.407,
    15: 3.472,
    16: 3.532,
    17:3.588,
    18:3.640,
    19:3.689,
    20:3.735,
}

In [44]:

def process_performance(df,num_sub_sample):
  #print('dim: ',name)
  n=num_sub_sample
  num_sample=n*25
  
  df_temp=df[-num_sample:]
  #print(len(df))
  #print(len(df_temp))
  usl=df.USL[0]
  lsl=df.LSL[0]
  m=df.Value.mean()

  #Ppk
  sigma=st.stdev(df_temp.Value)
  Pp = float(usl - lsl) / (6*sigma)
  Ppu = float(usl - m) / (3*sigma)
  Ppl = float(m - lsl) / (3*sigma)
  Ppk = np.min([Ppu, Ppl])
  

  #UCL, LCL, Mean
  k=3
  df['UCL']=df_temp['Value'].mean() + sigma*k
  df['LCL']=df_temp['Value'].mean() - sigma*k
  df['Mean']=df_temp['Value'].mean()

  #Cpk
  
  temp=df.groupby('Date').agg({'Value':['min','max']})
  temp['R']=temp['Value','max']-temp['Value','min']
  R=temp['R'].mean()

  if n <= 20:
    sigma_within = R/constants[n]
  else:
    sigma_within = R/constants[20]

  Cp = float(usl - lsl) / (6*sigma_within)
  Cpu = float(usl - m) / (3*sigma_within)
  Cpl = float(m - lsl) / (3*sigma_within)
  Cpk = np.min([Cpu, Cpl])
  #print('Pp:{:.2f} , Ppk: {:.2f}'.format(Pp,Ppk))
  #print('Cp:{:.2f} , Cpk:{:.2f}'.format(Cp,Cpk))
  Cp=round(Cp,2)
  Cpk=round(Cpk,2)
  Pp=round(Pp,2)
  Ppk=round(Ppk,2)
  return Cp,Cpk,Pp,Ppk

In [45]:
process_indicator_dict={}
process_indicator_df=pd.DataFrame(columns=['Dim','Cp','Pp','Cpk','Ppk'])
#process_indicator_df.columns=['Dim','Cp','Cpk','Pb','Ppk']
i=0
for dim_name in list(base_file.keys()):
  df=base_file[dim_name]
  Cp,Cpk,Pp,Ppk=process_performance(df,num_sub_sample)
  process_indicator_dict[dim_name]=[Cp,Pp,Cpk,Ppk]
  process_indicator_df.loc[i]=dim_name,Cp, Pp, Cpk, Ppk
  i+=1

process_indicator_df=process_indicator_df.sort_values(by='Cpk').reset_index(drop=True)

#conver process indicator dict to list name:
name_list=[]
for name in process_indicator_dict.keys():
  dim_name='dim: ' + name + ' Cp: ' + str(process_indicator_dict[name][0]) + ' Pp: '+ \
  str(process_indicator_dict[name][1]) +' Cpk: ' \
  + str(process_indicator_dict[name][2]) +' Ppk: '+ str(process_indicator_dict[name][3]) 
  name_list.append(dim_name)

In [46]:
process_indicator_df

,Dim,Cp,Pp,Cpk,Ppk
0,X,3.12,2.43,0.61,0.48
1,AI,1.23,0.75,1.02,0.63
2,AM,1.97,1.31,1.12,0.75
3,AK,2.52,1.74,1.18,0.82
4,AD,1.58,0.81,1.34,0.68
5,K-Pitch,2.09,1.93,1.73,1.60
6,AL,4.55,2.76,3.72,2.25


In [47]:
name_list

['dim: K-Pitch Cp: 2.09 Pp: 1.93 Cpk: 1.73 Ppk: 1.6',
 'dim: AM Cp: 1.97 Pp: 1.31 Cpk: 1.12 Ppk: 0.75',
 'dim: AK Cp: 2.52 Pp: 1.74 Cpk: 1.18 Ppk: 0.82',
 'dim: AI Cp: 1.23 Pp: 0.75 Cpk: 1.02 Ppk: 0.63',
 'dim: AL Cp: 4.55 Pp: 2.76 Cpk: 3.72 Ppk: 2.25',
 'dim: X Cp: 3.12 Pp: 2.43 Cpk: 0.61 Ppk: 0.48',
 'dim: AD Cp: 1.58 Pp: 0.81 Cpk: 1.34 Ppk: 0.68']

In [48]:
def hightlight_price(row):
    ret = ["" for _ in row.index]
    if row.Cpk < limit or row.Ppk < limit:
      ret[row.index.get_loc("Dim")] = "background-color: darkorange"
    if row.Cpk < limit:    
      ret[row.index.get_loc("Cpk")] = "background-color: darkorange"
    if row.Ppk < limit:  
      ret[row.index.get_loc("Ppk")] = "background-color: darkorange"
    return ret

limit = 1.33  # sigma: 4, Yield: 99.99%   
process_indicator_df.style.apply(hightlight_price, axis=1)

,Dim,Cp,Pp,Cpk,Ppk
0,X,3.120000,2.430000,0.610000,0.480000
1,AI,1.230000,0.750000,1.020000,0.630000
2,AM,1.970000,1.310000,1.120000,0.750000
3,AK,2.520000,1.740000,1.180000,0.820000
4,AD,1.580000,0.810000,1.340000,0.680000
5,K-Pitch,2.090000,1.930000,1.730000,1.600000
6,AL,4.550000,2.760000,3.720000,2.250000


# Line chart

In [49]:
df_dict=base_file
fig = make_subplots(          # Dim name
    rows=len(df_dict), cols=1,
    #shared_xaxes=True, # share same axis
    vertical_spacing=0.05, # adjust spacing between charts
    #column_widths=[0.8, 0.2],
    #subplot_titles=(list(df_dict.keys()))
    subplot_titles=(name_list)
)
i=1
for name in list(df_dict.keys()): #also group
  df=df_dict[name]
  df=df.sort_values(by=['Date'])
  for a in df.columns[1:]:
    df[a] = df[a].round(decimals=3)
  df=df.reset_index(drop=True)
  df_group=df.groupby('Date').mean()
  #df=df.set_index('Date')
  #if start_date != '' and end_date != '':
  #  df=df[start_date:end_date] 
  #Control chart 1 
  fig.append_trace(go.Scatter(
                          x=df_group.index, y=df_group['Value'],mode='lines+markers',name='Mean '+name,
                          line=dict( color='#4280F5')
                          ),row=i, col=1)
  #USL, LSL
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['USL'],name='USL '+name, line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['LSL'],name='LSL '+name,line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df['Datef'], y=df['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
  # UCL, LCL
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['UCL'],name='UCL '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['LCL'],name='LCL '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['Mean'],name='Mean '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  i=i+1


fig.update_layout(height=200*len(df_dict), width=1200, title_text=material_name)
#fig update each process (contain a lot of dim inside)
fig.show()
 

# Box plot chart

In [50]:
df_dict=base_file
fig = make_subplots(          # Dim name
    rows=len(df_dict), cols=1,
    #shared_xaxes=True, # share same axis
    vertical_spacing=0.05, # adjust spacing between charts
    #column_widths=[0.8, 0.2],
    #subplot_titles=(list(df_dict.keys()))
    subplot_titles=(name_list)
)
i=1
for name in list(df_dict.keys()): #also group
  df=df_dict[name]
  df=df.sort_values(by=['Date'])
  for a in df.columns[1:]:
    df[a] = df[a].round(decimals=3)
  df=df.reset_index(drop=True)
  df=df.set_index('Date')
  #df=df.set_index('Date')
  #if start_date != '' and end_date != '':
  #  df=df[start_date:end_date]
  #Control chart 1 
  fig.append_trace(go.Box(
                          x=df.index, y=df['Value'],name='value '+name,
                          line=dict( color='#4280F5')
                          ),row=i, col=1)
  #USL, LSL
  fig.append_trace(go.Scatter(x=df.index, y=df['USL'],name='USL '+name, line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df.index, y=df['LSL'],name='LSL '+name,line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df['Datef'], y=df['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
  i=i+1


fig.update_layout(height=200*len(df_dict), width=1200, title_text=material_name)
#fig update each process (contain a lot of dim inside)
fig.show()
 

# Forecast

In [ ]:
from fbprophet import Prophet
predict_future=3 # Month

In [ ]:

#@st.cache(suppress_st_warning=True)
#def predict_prophet(DFdict_final):
DFdict_final=base_file
DF_predict_final_={} # only contail predict
#DF_predict_all_final={} # contain predict and past value
for name in DFdict_final.keys(): #also group
  print(name)
  df_temp_2=DFdict_final[name] # gia tri mean thong thuong
  df_temp_2=df_temp_2.groupby('Date').mean()
  df_predict=df_temp_2.copy()
  #df_predict=df_predict.resample('1M').ffill() # gia tri mean da resample
  df_predict=df_predict.resample('M').mean().dropna()
  df_fb=df_predict.reset_index()#
  df_fb=df_fb[[df_fb.columns[0],df_fb.columns[1]]]
  df_fb.columns=['ds','y']

  # Prophet will by default fit weekly and yearly seasonalities,
  model = Prophet( #yearly_seasonality=True
                ) #instantiate Prophet with only yearly seasonality as our data is monthly 
  model.fit(df_fb) #fit the model with your dataframe

  # predict for five months in the furure and MS - month start is the frequency
  future = model.make_future_dataframe(periods = predict_future, freq = 'M')  
  # now lets make the forecasts
  forecast = model.predict(future)
  new_forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
  new_forecast['USL']=df_temp_2['USL'][0]
  new_forecast['LSL']=df_temp_2['LSL'][0]
  #new_forecast['Nominal']=df_temp_2['Mean'][0]
  new_forecast['UCL']=df_temp_2['UCL'][0]
  new_forecast['LCL']=df_temp_2['LCL'][0]
  new_forecast['mean']=df_temp_2['Mean'][0]
  DF_predict_final_[name]=new_forecast 

In [ ]:
#@st.cache(suppress_st_warning=True,allow_output_mutation=True)
#def line_chart_prophet(DFdict_final,DF_predict_final):

DFdict_final=base_file
DF_predict_final=DF_predict_final_
i=1
#Layout
fig = make_subplots(
    rows=len(DFdict_final), cols=1,
    #shared_xaxes=True,
    vertical_spacing=0.05,
    #column_widths=[0.8, 0.2],
    subplot_titles=(name_list)
)

for name in DFdict_final.keys(): #also group
  df=DFdict_final[name].copy()
  df=df.sort_values(by=['Date'])
  for a in df.columns[1:]:
    df[a] = df[a].round(decimals=3)
  df=df.groupby('Date').mean()
  #Control chart 1 
  fig.append_trace(go.Scatter(
                          x=df.index, y=df['Value'],
                          mode='lines+markers',
                          name='mean ' + name,line=dict( color='#4280F5')
                          ),row=i, col=1
                )
  #Predict:
  df_predict=DF_predict_final[name].copy()
  fig.append_trace(go.Scatter(
                      x=df_predict['ds'], y=df_predict['yhat'],
                      mode='lines+markers',
                      name='mean predict ' + name,line=dict( color='#060891')
                      ),row=i, col=1
            )

  #USL, LSL
  fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['USL'],name='USL '+name, line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['LSL'],name='LSL '+name,line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
  # UCL, LCL
  #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['UCL'],name='UCL '+name, line=dict( color='#33C2FF')),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['LCL'],name='LCL '+name, line=dict( color='#33C2FF')),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['mean'],name='mean '+name, line=dict( color='#33C2FF')),row=i, col=1)
  # Fill between:
  fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['yhat_upper'],name='yhat_upper '+name, line=dict( color='#9FA0F8'),fill=None,mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['yhat_lower'],name='yhat_lower '+name, line=dict( color='#9FA0F8'), fill='tonexty',mode='lines'),row=i, col=1)


  #Final layout:
  #fig.update_layout(height=400, width=1400, title_text='Line chart'+name)
  i=i+1
  
fig.update_layout(height=200*len(DFdict_final), width=1200, title_text=material_name)
fig.show()


In [ ]:
df